In [1]:
import sys
import csv
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np
import pickle 
import pprint
import requests
from datetime import date, datetime
from random import *
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import os
import datetime
import shutil
import glob

In [2]:
pp = pprint.PrettyPrinter(indent=4)

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('window-size=1000x500')
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("disable-gpu")

In [9]:
chrome_path = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(chrome_path, options=options)

url = 'https://www.gocamping.or.kr/bsite/camp/info/list.do?pageUnit=2500&searchKrwd=&listOrdrTrget=last_updusr_pnttm&pageIndex=1'
driver.get(url) 

time.sleep(10)

source = driver.page_source 
data = bs(source, 'html.parser')

In [72]:
cont = data.findAll("div",{"class":"camp_cont"})

## camping site 수집

In [168]:
cols = ['name','url','addr']
df = pd.DataFrame(columns=cols)
prefix = 'https://www.gocamping.or.kr'
for i in range(len(cont)):
    name = cont[i].h2.a.get_text().split("]")[1]
    url_t = cont[i].h2.a['href'].split("&")[0]
    url = ''.join([prefix,url_t])
    addr = cont[i].ul(class_="addr")[0].get_text()
    
    info = [name, url, addr]
    dict1 = dict(zip(cols, info))
    df = df.append(dict1, ignore_index=True)

df['name'] = df.name.str.strip()
df['hashtag'] = df.name.str.replace(' ','')
df['hashtag'] = df.hashtag.str.replace('(주)','')

In [169]:
df.head()

,name,url,addr,hashtag
0,아트앤오토캠핑장,https://www.gocamping.or.kr/bsite/camp/info/re...,울산광역시 울주군 상북면 이천리 700,아트앤오토캠핑장
1,지구캠프,https://www.gocamping.or.kr/bsite/camp/info/re...,강원 영월군 무릉도원면 무릉법흥로 1104,지구캠프
2,힐링스토리,https://www.gocamping.or.kr/bsite/camp/info/re...,강원 횡성군 서원면 금대1길 24-68 힐링스토리,힐링스토리
3,밀양댐오토캠핑장,https://www.gocamping.or.kr/bsite/camp/info/re...,경상남도 밀양시 단장면 고례리 1512-1번지,밀양댐오토캠핑장
4,별볼일 있는 야영장,https://www.gocamping.or.kr/bsite/camp/info/re...,전라남도 보성군 문덕면 양동리 942,별볼일있는야영장


## instagram hashtag 수집

In [187]:
def post_count(df, tag_nm, num=0):
    rows_list = []
    tag_list = df[tag_nm].values.tolist()
    
    today = datetime.datetime.today() 
    today_ymd = today.strftime('%Y%m%d')

    path = '../file/' 
    new_path = ''.join([path, today_ymd,'/'])
    file_nm = '_'.join(['camping', today_ymd])
    
    output_nm = ''.join([path, file_nm, ".csv"])
    mergeoutput_nm = ''.join([new_path, file_nm, ".csv"])
    
    cols = list(df.columns) + ['num_posts','date_time']
    df_out = pd.DataFrame(columns=cols)

    try:
        os.mkdir(new_path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)
    
    chrome_path = '/usr/local/bin/chromedriver'
    driver = webdriver.Chrome(chrome_path, options=options)

    for i in range(num,len(tag_list)):
        driver.get('https://www.instagram.com/explore/tags/'+tag_list[i]+'/?hl=en') 
        time.sleep(2) 
        source = driver.page_source 
        data = bs(source, 'html.parser') 
        tag = data.find("span",{"class": "g47SY "}) 
        try: 
            val = tag.text 
            val = int(val.replace(',' , '')) 
        except AttributeError: 
            val = 0

        info = [df.name[i], df.url[i], df.addr[i], df.hashtag[i], val, today]
        dict1 = dict(zip(cols,info))
        df_out = df_out.append(dict1, ignore_index=True)
        
#         time.sleep(0.2)

        if i>0 and i%50 == 0:
            print(df_out.tail(5))
            driver.quit()
            df_out.to_csv(new_path+file_nm+"_"+f"{i:0>5}"+'.csv', index=False, header=False)
            
            df_out = pd.DataFrame(columns=cols)
            
            time.sleep(3)
            
            driver = webdriver.Chrome(chrome_path, options=options)

    df_out.to_csv(new_path+file_nm+"_"+f"{i:0>5}"+'.csv', index=False, header=False)
    driver.quit()
    
    with open(output_nm, 'wb') as outfile:
        for filename in glob.glob(new_path+file_nm+'*.csv'):
    #         print(filename)
            if filename == output_nm:
                # don't want to copy the output into the output
                continue

            with open(filename, 'rb') as readfile:
                readfile.readline()
                shutil.copyfileobj(readfile, outfile)

    outfile = pd.read_csv(output_nm, names=cols, index_col='no')
    print(outfile.head(10))
    outfile.to_csv(output_nm)

In [188]:
post_count(df, 'hashtag')

Creation of the directory ../file/ failed
       name                                                url  \
46  느티나무야영장  https://www.gocamping.or.kr/bsite/camp/info/re...   
47     에이제이  https://www.gocamping.or.kr/bsite/camp/info/re...   
48  횡성초보캠핑장  https://www.gocamping.or.kr/bsite/camp/info/re...   
49   황학동캠핑장  https://www.gocamping.or.kr/bsite/camp/info/re...   
50      호연재  https://www.gocamping.or.kr/bsite/camp/info/re...   

                         addr  hashtag num_posts                  date_time  
46     경상남도 산청군 삼장면 유평리  233   느티나무야영장        29 2020-06-11 17:26:38.517792  
47  경기도 남양주시 수동면 내방리  330-10      에이제이      1624 2020-06-11 17:26:38.517792  
48   강원도 횡성군 서원면 압곡리  298-68   횡성초보캠핑장       221 2020-06-11 17:26:38.517792  
49    강원도 원주시 신림면 구학리  617-2    황학동캠핑장         0 2020-06-11 17:26:38.517792  
50      전라북도 완주군 소양면 대흥리 478       호연재      1175 2020-06-11 17:26:38.517792  
           name                                                url  \
45       깊은골캠핑장  https:

ValueError: Index no invalid

In [193]:
len(df)
df.head()

,name,url,addr,hashtag
0,아트앤오토캠핑장,https://www.gocamping.or.kr/bsite/camp/info/re...,울산광역시 울주군 상북면 이천리 700,아트앤오토캠핑장
1,지구캠프,https://www.gocamping.or.kr/bsite/camp/info/re...,강원 영월군 무릉도원면 무릉법흥로 1104,지구캠프
2,힐링스토리,https://www.gocamping.or.kr/bsite/camp/info/re...,강원 횡성군 서원면 금대1길 24-68 힐링스토리,힐링스토리
3,밀양댐오토캠핑장,https://www.gocamping.or.kr/bsite/camp/info/re...,경상남도 밀양시 단장면 고례리 1512-1번지,밀양댐오토캠핑장
4,별볼일 있는 야영장,https://www.gocamping.or.kr/bsite/camp/info/re...,전라남도 보성군 문덕면 양동리 942,별볼일있는야영장


In [196]:
df2 = pd.read_csv('../file/camping_20200618.csv', columns=cols)
df2.head()

TypeError: parser_f() got an unexpected keyword argument 'columns'